# An RNN model for temperature data
Multi-site model

In [ ]:
import math
import sys
import numpy as np
import utils_batching
import tensorflow as tf
print("Tensorflow version: " + tf.__version__)

In [ ]:
from matplotlib import pyplot as plt
import utils_prettystyle 

## Hyperparameters

In [ ]:
RESAMPLE_BY = 5     # averaging period in days (training on daily data is too much)
RNN_CELLSIZE = 100  # size of the RNN cells
NLAYERS = 2         # number of stacked RNN cells (needed for tensor shapes but code must be changed manually)
SEQLEN = 100        # unrolled sequence length
BATCHSIZE = 32      # mini-batch size
DROPOUT_PKEEP = 0.7 # probability of neurons not being dropped (should be between 0.5 and 1)
ACTIVATION = tf.nn.tanh # Activation function for GRU cells (tf.nn.relu or tf.nn.tanh)

## Load temperatures

In [ ]:
#32 weather stations
weather_stations = ['USC00010655.csv','USC00012813.csv','USC00016478.csv','USC00020949.csv','USC00021314.csv',
'USC00025635.csv','USC00026468.csv','USC00029359.csv','USC00030458.csv',
'USC00031948.csv','USC00032794.csv','USC00032930.csv','USC00033466.csv','USC00033821.csv','USC00033862.csv',
'USC00036920.csv','USC00036928.csv','USC00040343.csv','USC00041244.csv','USC00041428.csv',
'USC00042598.csv','USC00042713.csv','USC00042934.csv','USC00043551.csv','USC00043875.csv','USC00044957.csv',
'USC00045118.csv','USC00046074.csv','USC00046136.csv','USC00046730.csv','USC00047150.csv','USC00047916.csv']

# possible evaluation files: temperatures/USC00040343.csv"
with tf.gfile.Open("gs://ml1-demo-martin/good_temperature_data/USC00040343.csv", mode='rb') as f:
    eval_temperatures = np.genfromtxt(f, delimiter=",", skip_header=True,
                      usecols=[0,1,2,3], converters = {0: lambda s: np.datetime64(s)})

In [ ]:
# You can adjust the visualisation range here
# Interpolated regions of the dataset are marked in red
visu_temperatures = eval_temperatures[:]

# when reading from CSV, numpy names its columns f0, f1, f2, ...
dates = visu_temperatures[:]['f0']
min_temps = visu_temperatures[:]['f1']
max_temps = visu_temperatures[:]['f2']
interpolated = visu_temperatures[:]['f3']

interpolated_sequence = False
#plt.plot(dates, max_temps)
for i, date in enumerate(dates):
    if interpolated[i]:
        if not interpolated_sequence:
            startdate = date
        interpolated_sequence = True
        stopdate=date
    else:
        if interpolated_sequence:
            # light shade of red just for visibility
            plt.axvspan(startdate+np.timedelta64(-5, 'D'), stopdate+np.timedelta64(6, 'D'), facecolor='#FFCCCC', alpha=1)
            # actual interpolated region
            plt.axvspan(startdate+np.timedelta64(-1, 'D'), stopdate+np.timedelta64(1, 'D'), facecolor='#FF8888', alpha=1)
        interpolated_sequence = False
plt.fill_between(dates, min_temps, max_temps).set_zorder(10)
plt.show()

## Resampling

In [ ]:
# the dataset we will be working with
evaldata = eval_temperatures[:]['f2'] # max temperatures
rounded_data_len = evaldata.shape[0]//RESAMPLE_BY*RESAMPLE_BY
evaldata = np.reshape(evaldata[:rounded_data_len], [-1, RESAMPLE_BY])
evaldata = np.mean(evaldata, axis=1)

In [ ]:
plt.plot(evaldata[:365*5//RESAMPLE_BY]) # display five years worth of data
plt.show()

## Build dataset

In [ ]:
traindata = []
for station_i in range(BATCHSIZE):
    with tf.gfile.Open("gs://ml1-demo-martin/good_temperature_data/"+weather_stations[station_i], mode='rb') as f:
        temperatures = np.genfromtxt(f, delimiter=",",
                                     skip_header=True,
                                     usecols=[0,1,2,3],
                                     converters = {0: lambda s: np.datetime64(s)})
        temperatures = temperatures[:]['f2'] # max temperatures
        traindata.append(temperatures)
data = np.stack(traindata, axis=0)
print(data.shape)

## Visualize training sequences
This is what the neural network will see during training.

In [ ]:
# The function rnn_sampling_sequencer puts one weather station per line in a batch
# and continues with data from the same station in corresponding lines in the next batch.
subplot = 231
for samples, labels, epoch in utils_batching.rnn_sampling_sequencer(data, RESAMPLE_BY, SEQLEN, nb_epochs=1):
    plt.subplot(subplot)
    plt.plot(samples[0,:])
    subplot += 1
    if subplot==237: break
print("Sample shape: " + str(samples.shape))
print("Label shape: " + str(labels.shape))
print("First samples:")
plt.show()

## The model definition
When executed, this function instantiates the Tensorflow graph for our model.
![deep RNN schematic](images/deep_rnn.png)

In [ ]:
def model_rnn_fn(features, Hin, labels, dropout_pkeep):
    X = features
    batchsize = tf.shape(X)[0]
    seqlen = tf.shape(X)[1]
    
    cells = [tf.nn.rnn_cell.GRUCell(RNN_CELLSIZE, activation=ACTIVATION) for _ in range(NLAYERS)]
    # dropout useful between cell layers only: no output dropout on last cell
    dcells = [tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob = dropout_pkeep) for cell in cells[:-1]]
    dcells.append(cells[-1])
    # a stacked RNN cell still works like an RNN cell
    cell = tf.nn.rnn_cell.MultiRNNCell(dcells, state_is_tuple=False)
    # X[BATCHSIZE, SEQLEN, 1], Hin[BATCHSIZE, RNN_CELLSIZE*NLAYERS]
    # the sequence unrolling happens here
    Yn, H = tf.nn.dynamic_rnn(cell, X, initial_state=Hin, dtype=tf.float32)
    # Yn[BATCHSIZE, SEQLEN, RNN_CELLSIZE]
    Yn = tf.reshape(Yn, [batchsize*seqlen, RNN_CELLSIZE])
    Yr = tf.layers.dense(Yn, 1) # Yr [BATCHSIZE*SEQLEN, 1]
    Yr = tf.reshape(Yr, [batchsize, seqlen, 1]) # Yr [BATCHSIZE, SEQLEN, 1]
    Yout = Yr[:,-1,:] # Last output Yout [BATCHSIZE, 1]
    
    loss = tf.losses.mean_squared_error(Yr, labels) # labels[BATCHSIZE, SEQLEN, 1]
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(loss)
    
    return Yout, H, loss, train_op, Yr

## Instantiate the model

In [ ]:
# placeholder for inputs
Hin = tf.placeholder(tf.float32, [None, RNN_CELLSIZE * NLAYERS])
samples = tf.placeholder(tf.float32, [None, None, 1]) # [BATCHSIZE, SEQLEN, 1]
labels = tf.placeholder(tf.float32, [None, None, 1]) # [BATCHSIZE, SEQLEN, 1]
dropout_pkeep = tf.placeholder(tf.float32)

# instantiate the model
Yout, H, loss, train_op, Yr = model_rnn_fn(samples, Hin, labels, dropout_pkeep)

## Inference
This is a generative model: run one trained RNN cell in a loop

In [ ]:
def prediction_run(prime_data, run_length):
    H_ = np.zeros([1, RNN_CELLSIZE * NLAYERS]) # zero state initially
    Yout_ = np.zeros([1, 1])
    data_len = prime_data.shape[0]

    # prime the state from data
    if data_len > 0:
        Yin = np.array(prime_data)
        Yin = np.reshape(Yin, [1, data_len, 1]) # reshape as one sequence
        feed = {Hin: H_, samples: Yin, dropout_pkeep: 1.0} # no dropout during inference
        Yout_, H_ = sess.run([Yout, H], feed_dict=feed)
    
    # run prediction
    # To generate a sequence, run a trained cell in a loop passing as input and input state
    # respectively the output and output state from the previous iteration.
    results = []
    for i in range(run_length):
        Yout_ = np.reshape(Yout_, [1, 1, 1]) # batch of a single sequence of a single vector with one element
        feed = {Hin: H_, samples: Yout_, dropout_pkeep: 1.0} # no dropout during inference
        Yout_, H_ = sess.run([Yout, H], feed_dict=feed)
        results.append(Yout_[0,0])
        
    return np.array(results)

## Initialize Tensorflow session
This resets all neuron weights and biases to initial random values

In [ ]:
# first input state
Hzero = np.zeros([BATCHSIZE, RNN_CELLSIZE * NLAYERS])
# variable initialization
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run([init])

## The training loop
You can re-execute this cell to continue training

In [ ]:
NB_EPOCHS = 300

H_ = Hzero
losses = []
indices = []
last_epoch = 0
for i, (next_samples, next_labels, epoch) in enumerate(utils_batching.rnn_sampling_sequencer(data, RESAMPLE_BY, SEQLEN, nb_epochs=NB_EPOCHS)):
    next_samples = np.expand_dims(next_samples, axis=2) # model wants 3D inputs [BATCHSIZE, SEQLEN, 1] 
    next_labels = np.expand_dims(next_labels, axis=2)
    
    # reinintialize state between epochs
    #if epoch != last_epoch:
    #    H_ = Hzero

    feed = {Hin: H_, samples: next_samples, labels: next_labels, dropout_pkeep: DROPOUT_PKEEP}
    Yout_, H_, loss_, _, Yr_ = sess.run([Yout, H, loss, train_op, Yr], feed_dict=feed)
    last_epoch = epoch
    
    # print progress
    if i%30 == 0:
        print("epoch " + str(epoch) + ", batch " + str(i) + ", loss=" + str(np.mean(loss_)))
        sys.stdout.flush()
    if i%10 == 0:
        losses.append(np.mean(loss_))
        indices.append(i)
# This visualisation can be helpful to see how the model "locks" on the shape of the curve
#    if i%100 == 0:
#        plt.figure(figsize=(10,2))
#        plt.plot(next_labels[0,:,0])
#        plt.plot(Yr_[0,:,0])
#        plt.show()

In [ ]:
plt.ylim(ymax=np.amax(losses[1:])) # ignore first value for scaling
plt.plot(indices, losses)
plt.show()

In [ ]:
print(evaldata.shape)

In [ ]:
PRIMELEN=1000
RUNLEN=500
OFFSET=300
RMSELEN=128
prime_data = evaldata[OFFSET:OFFSET+PRIMELEN]
results = prediction_run(prime_data, RUNLEN)

In [ ]:
disp_data = evaldata[OFFSET:OFFSET+PRIMELEN+RUNLEN]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.subplot(211)
plt.text(PRIMELEN,2.5,"DATA |", color=colors[1], horizontalalignment="right")
plt.text(PRIMELEN,2.5,"| PREDICTED", color=colors[0], horizontalalignment="left")
displayresults = np.ma.array(np.concatenate((np.zeros([PRIMELEN]), results)))
displayresults = np.ma.masked_where(displayresults == 0, displayresults)
plt.plot(displayresults)
displaydata = np.ma.array(np.concatenate((prime_data, np.zeros([RUNLEN]))))
displaydata = np.ma.masked_where(displaydata == 0, displaydata)
plt.plot(displaydata)
plt.subplot(212)
plt.text(PRIMELEN,2.5,"DATA |", color=colors[1], horizontalalignment="right")
plt.text(PRIMELEN,2.5,"| +PREDICTED", color=colors[0], horizontalalignment="left")
plt.plot(displayresults)
plt.plot(disp_data)
plt.axvspan(PRIMELEN, PRIMELEN+RMSELEN, color='grey', alpha=0.1, ymin=0.05, ymax=0.95)
plt.show()

In [ ]:
rmse = math.sqrt(np.mean((evaldata[OFFSET+PRIMELEN:OFFSET+PRIMELEN+RMSELEN] - results[:RMSELEN])**2))
print("RMSE on {} predictions (shaded area): {}".format(RMSELEN, rmse))
sys.stdout.flush()

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
[http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.